In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy

nlp = spacy.load("en_core_web_lg")

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [6]:
df = pd.read_csv("../data/raw/train.csv")

In [81]:
import contractions
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import names
from textacy import preprocessing

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('names', quiet=True)

lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words("english"))
stops.remove("not")
stops.remove("won")

human_names = set(names.words("male.txt") + names.words("female.txt"))
human_names.remove("Happy")

def preprocess(text):
    text = preprocessing.normalize.quotation_marks(text)
    # Resolve contractions & slangs
    text = contractions.fix(text, slang=True)
    # Remove username handles
    # text = re.sub(r"@[^\s]+", "", text)
    text = preprocessing.replace.user_handles(text, "")
    text = re.sub(r"_[^\s]+", "", text)
    
    text = preprocessing.replace.numbers(text, "")
    text = preprocessing.replace.urls(text, "")
    text = preprocessing.remove.html_tags(text)
    text = preprocessing.remove.punctuation(text)
    text = preprocessing.normalize.whitespace(text)
    
    # TODO: filter out names & certain stopwords
    # TODO: ugly to tokenize in the preprocess fn..
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in human_names]
    tokens = [token for token in tokens if token.lower() not in stops]
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]
    text = " ".join(tokens)
    
    text = text.lower()
    return text
preprocess("wouldnt")

'would not'

In [82]:
df["preproc_text"] = df["text"].apply(preprocess)

In [29]:
from tqdm.auto import tqdm  # for notebooks
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [84]:
X = []
for doc in tqdm(nlp.pipe(df["preproc_text"], disable=[ "tagger", "parser","senter", "attribute_ruler", "lemmatizer", "ner"]), total=df.shape[0]):
    X.append(doc.vector)
# X = df["preproc_text"].progress_apply(lambda text: nlp(text).vector)
X = np.vstack(X)

  0%|          | 0/24732 [00:00<?, ?it/s]

In [47]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(df["sentiment"])

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(alpha=1e-4)

n_epochs = 100
batch_size = 1024
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

pbar =  tqdm(range(n_epochs))
for _ in pbar:
    for batch in tqdm(iterate_minibatches(X_train, y_train, batch_size, shuffle=True), leave=False, disable=True):
        x_batch, y_batch = batch
        clf.partial_fit(x_batch, y_batch, classes=[0,1, 2])
        
    y_pred = clf.predict(X_test)
    pbar.set_postfix({'accuracy': accuracy_score(y_test, y_pred)})


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [87]:
len(list(iterate_minibatches(X_train, y_train, batch_size, shuffle=True)))

19

In [88]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.6696988073579947

In [51]:
import textacy
textacy.preprocessing.replace.user_handles("@ast @_straneii _antna", "")

'  _antna'